In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
import time
import math
import matplotlib as mp
import scipy as sp
import pylab as py

# SPH Implementation
### The Gaussian Smoothing function is used to smooth out the point values, extrapolating them to other positions.
 Given the distribution of particles, we can reconstruct the density at any location using the smoothing kernels. For example, the density at each SPH particle location is a sum over all the particles with weighting set by the distances between particles and the smoothing kernel:

In [16]:
def grad_cubic_spline_kernel(x,y,z,h):
    sigma = 10/(7*np.pi*h**2)
    r = np.sqrt(x**2 + y**2 + z**2)
    if r <= 1:
        return -1.5*r + 0.75*r**2*sigma
    elif r<=2:
        return -0.75*sigma*(2-r)**2
    else:
        return 0

def W(x,y,z,h, tag = "gaussian"):
    """ Defined Smoothing kernel (3D).
    (x is a vector/matrix of x positions, y is a vector/matrix of y positions. 
    z = vector/matrix of z positions, h = smoothing length, w = evaluated smoothing function.
    Implemented Gaussian, Wendland C2 and Wendland C4 smoothing kernels as recommended by (Rosswog, 2015).
    """
    r = np.sqrt(x**2 + y**2 + z**2) 
    ### ========= Gaussian kernel
    if tag == "gaussian":
        w = (1.0 / (h*np.sqrt(np.pi)))**3 * np.exp( -r**2 / h**2) # return w  
    ### ========= WENDLAN KERNEL (ROSSWOG, 2015) ========= ###    # bb
    elif tag == "wendland_2": # Wendland C2
        alpha = 21/(16*np.pi*h**3)
        if r <= 2:
            w = alpha*(1-r/2)**4*(2*r+1)
        else:
            w = 0         
    elif tag == "wendland_4": # Wendland C4
        alpha = 495/(256*np.pi*h**3)
        if r <= 2:
            w = alpha*(1-r/2)**6*(35/12*r**2 + 3*r + 1)
        else:
            w = 0
    elif tag == "cubic_spline":
        return grad_cubic_spline_kernel(x,y,z,h)
    else:
        raise NotImplementedError("Trying to implement Unknown Kernel.")
    return w

def gradW(x,y,z,h,tag = "gaussian"):
    """ Gradient of smoothing kernel. We can reconstruct the density at any location using the smoothing kernels.
    (x is a vector/matrix of x positions, y is a vector/matrix of y positions, z is a vector/matrix of z positions, h is the smoothing length, wx, wy, wz  is the evaluated gradient)"""
    r = np.sqrt(x**2 + y**2 + z**2)  
    if tag == "gaussian":
        n = -2 * np.exp( -r**2 / h**2) / h**5 / (np.pi)**(3/2)
    ### ========= WENDLAN KERNEL (ROSSWOG, 2015) ========= ###  
        # Gradient of WendlandQuintic kernel(C2) for 3D.         # https://pysph.readthedocs.io/en/latest/reference/kernels.html
    elif tag == "wendland_2":
        alpha = 7/(4*np.pi*h**2) ## Take derivative wrt r
        r[r>2] = 0 # set all the values in r greater than 2 to 2.
        n = alpha*(-2*(1-r/2)**3*(2*r+1) +2*(1-r/2)**4)
        # Gradient of WendlandQuintic kernel(C4) for 3D.          # https://pysph.readthedocs.io/en/latest/reference/kernels.html
    elif tag == "wendland_4": # Wendland C4
        r[r>2] = 0
        alpha = 495/(256*np.pi*h**3) ## Take derivative wrt r
        n = alpha*-2.5*(1-r/2)**5*(35/12*r**2 + 3*r + 1) + (1-r/2)**6*(35/12*r +3) ## Take derivative wrt r
    elif tag == "cubic_spline":
        return grad_cubic_spline_kernel(x,y,z,h)
    else:
        raise NotImplementedError("Trying to use the gradient of an Unknown Kernel.")
    return n*x, n*y, n*z  # (gradient in the x, y, and z directions)
    
def getPairwiseSeparations(ri, rj):
    """ Just finds Cartesian Pairwise Separations between 2 points. 
    ri is an M x 3 matrix of positions, 
    rj    is an N x 3 matrix of positions 
    returns dx, dy, dz   are M x N matrices of separations. """
    M = ri.shape[0]
    N = rj.shape[0]
    rix = ri[:,0].reshape((M,1)) # positions ri = (x,y,z)
    riy = ri[:,1].reshape((M,1))
    riz = ri[:,2].reshape((M,1))
    rjx = rj[:,0].reshape((N,1)) # other set of points positions rj = (x,y,z)
    rjy = rj[:,1].reshape((N,1))
    rjz = rj[:,2].reshape((N,1))
    return rix - rjx.T , riy - rjy.T, riz - rjz.T # (dx, dy, dz) # return matrices that store all pairwise particle separations: r_i - r_j

def getDensity(r, pos, m, h ):
    """ Reconstruct the density at any location based on the sph points.
            r     is an M x 3 matrix of sampling locations
            pos   is an N x 3 matrix of SPH particle positions
          
          dx, dy, dz are all M x N matrices. (Seperations). 
          - Sum over all particle interactions with the points to get the density at the locations in r.
          
          m = particle mass, h = smoothing length """
    M = r.shape[0]
    dx, dy, dz = getPairwiseSeparations( r, pos );
    rho = np.sum( m * W(dx, dy, dz, h), 1 ).reshape((M,1)) # (56)
    return rho

In [20]:
r =  np.asarray([[0.8607446,0.8607446,0.8607446]])
pos =  np.asarray([[0.860744,0,1]])
m =  0.01
h = np.asarray([[11]])
getDensity(r, pos, m, h )

array([[1.34081344e-06]])

In [15]:





# Wendland kernel
h = 0.1
x = np.asarray([1])
y = np.asarray([1])
z = np.asarray([1])

out2 = gradW(x,y,z,h, tag = "wendland_2")

def w(q,h):
    # dim 3
    a_d = 21/(16*np.pi*h**3)
    return a_d*(1-q/2)**4*(2*q + 1)

def dw(x,h):
    a_d = 7/(4*np.pi*h**3)
    return a_d*-2*(1-x/2)**3*(2*x+1) + 2*(1-x/2)**4

r = np.sqrt(x**2+y**2+z**2)
out1 = dw(r,h)
print(grad_cubic_spline_kernel(x,y,z,h))
print(out2)
print(out1)


[-2.44860231]
(array([-1.16007529]), array([-1.16007529]), array([-1.16007529]))
[-8.96911677]


In [ ]:
def Vanilla_dv_dt1(m, M, P, rho,v,dWx,dWy,dWz,nu=0):
    """ Reconstruct dV/dt at any sph point.
             Vanilla Ice formulation, see Rosswog et. al (2015)

          dx, dy, dz are all M x N matrices. (Seperations). 
          - Sum over all particle interactions 
          
          m = particle mass, h = smoothing length """
    ##### Fix this part #####
    print("\n\n Inside vanilladvdt\n\n")
    Pa=P
    Pb = P
    rhoa = rho
    rhob = rho
    ########################
    dvdtx = np.sum(-m*(Pa/rhoa**2 + Pb/rhob**2)*dWx + nu*dWx, 1 ).reshape((M,1)) # pretty sure this is right. 
    dvdty = np.sum(-m*(Pa/rhoa**2 + Pb/rhob**2)*dWy + nu*dWx, 1 ).reshape((M,1))
    dvdtz = np.sum(-m*(Pa/rhoa**2 + Pb/rhob**2)*dWz + nu*dWx, 1 ).reshape((M,1))
    print("Vanilla: dvdt exmaple: ", str(dvdtx[0]))
    return dvdtx,dvdty,dvdtz

def RS_1(m, M, P, rho,v,dWx,dWy,dWz,nu=0):
    print("\n\n Inside RS1\n\n")
    """ Reconstruct dV/dt at any sph point.
             Vanilla Ice formulation, see Rosswog et. al (2015)

          dx, dy, dz are all M x N matrices. (Seperations). 
          - Sum over all particle interactions 
          
          m = particle mass, h = smoothing length """
    v_x,v_y,v_z = getPairwiseSeparations(v,v)
    print("RS1: Pairwise velocity separation exmaple: ", str(v_x[0]))
    vx_sum = v_x.sum(axis=0)
    print("VX_SUM: ", str(vx_sum))
    vy_sum = v_y.sum(axis=0)
    vz_sum = v_z.sum(axis=0)
    ########################
    dvdtx = np.sum(-m*(P/rho**2 + 0.5*nu)*v_x*dWx, 1 ).reshape((M,1))
    dvdty = np.sum(-m*(P/rho**2 + 0.5*nu)*v_y*dWy, 1 ).reshape((M,1))
    dvdtz = np.sum(-m*(P/rho**2  + 0.5*nu)*v_z*dWz, 1 ).reshape((M,1))
    return dvdtx,dvdty,dvdtz

def RS_2(gamma, rho, m,v, dWx,dWy,dWz,nu=0):
    """ Reconstruct dV/dt at any sph point.
             Vanilla Ice formulation, see Rosswog et. al (2015)

          dx, dy, dz are all M x N matrices. (Seperations). 
          - Sum over all particle interactions 
          
    m = particle mass, h = smoothing length """
    v_x, v_y, v_z = getPairwiseSeparations(v,v)
    v_ = np.sqrt(v_x**2 + v_y**2 + v_z**2)
    c = (gamma-1)*(1/rho**(gamma-1)) # constant, same dimensions as rho
    ########################
    dAdtx = np.sum(-m*(P/rho**2 + 0.5*nu)*v_*dWx, 1 ).reshape((M,1))
    dAdty = np.sum(-m*(P/rho**2 + 0.5*nu)*v_*dWy, 1 ).reshape((M,1))
    dAdtz = np.sum(-m*(P/rho**2  + 0.5*nu)*v_*dWz, 1 ).reshape((M,1))
    return dvdtx,dvdty,dvdtz

class discrete_lagrange():
    """ Lagrangian formulation """
    def __init__(self, tag = "vanilla"): 
        if tag == "vanilla":
            self.info = "Vanilla Ice discitization of lagrange formulations for conservation for an ideal fluid. Outlined in (10.1007/lrca-2015-1)."
            self.dvdt = lambda m, M, P, rho,v,dWx,dWy,dWz,nu : Vanilla_dv_dt1(m, M, P, rho,v,dWx,dWy,dWz,nu) 
        elif tag == "RS91":
            self.info = "Uses dudt from Rasio & Shaprio (1991)."
            self.dvdt = lambda m, M, P, rho,v,dWx,dWy,dWz,nu : RS_1(m, M, P, rho,v,dWx,dWy,dWz,nu) 
        elif tag == "RS92":
            self.info = "Uses dAdt from Rasio & Shaprio (1991)."
            self.dAdt = lambda gamma, rho, m, dWx,dWy,dWz,nu : RS_2(m, M, P, rho,v,dWx,dWy,dWz,nu) 